In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
USE DATABASE SOEMTHING;
CREATE SCHEMA IF NOT EXISTS raw_data;
CREATE SCHEMA IF NOT EXISTS analytics;

CREATE OR REPLACE FILE FORMAT csv_fmt
  TYPE = 'CSV'
  FIELD_DELIMITER = ','
  SKIP_HEADER = 1;


In [ ]:
USE SCHEMA raw_data;

CREATE OR REPLACE STAGE grades_stage
  FILE_FORMAT = csv_fmt;


In [ ]:
USE SCHEMA raw_data;

-- drop if exists, then load
CREATE OR REPLACE TABLE grades_raw (
  student_id VARCHAR,
  course_id  VARCHAR,
  grade      NUMBER
);

COPY INTO grades_raw
  FROM @grades_stage/grades.csv
  ON_ERROR = 'ABORT_STATEMENT';

In [ ]:
SHOW STAGES LIKE 'GRADES_STAGE';


In [ ]:
SELECT * FROM raw_data.grades_raw;


In [ ]:
USE SCHEMA analytics;

CREATE OR REPLACE TABLE student_grades AS
SELECT
  student_id,
  course_id,
  grade,
  CASE
    WHEN grade >= 90 THEN 'A'
    WHEN grade >= 80 THEN 'B'
    WHEN grade >= 70 THEN 'C'
    WHEN grade >= 60 THEN 'D'
    ELSE 'F'
  END AS letter_grade
FROM raw_data.grades_raw;


In [ ]:
SELECT * FROM analytics.student_grades LIMIT 10;


In [ ]:
USE SCHEMA analytics;

CREATE OR REPLACE VIEW avg_grade_per_course AS
SELECT
  course_id,
  ROUND(AVG(grade),2) AS avg_numeric,
  COUNT(*)            AS num_students
FROM student_grades
GROUP BY course_id
ORDER BY course_id;


In [ ]:
SELECT * FROM analytics.avg_grade_per_course;
